In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
np.set_printoptions(suppress=True)

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

y_input = tf.keras.utils.to_categorical(y_train)
x_input = (np.reshape(x_train, (x_train.shape[0], 784)) / 255.0).astype(np.float32)

In [2]:
W = tf.Variable(tf.zeros([784,10]),name='W')
b = tf.Variable(tf.zeros([10]),name='b')

# predicted probability for each class
def y_pred(x):
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
@tf.function
def loss(x,y):
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))



2023-05-18 08:09:45.448950: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 08:09:46.024576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10794 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


In [8]:
# Set up logging.
logdir = '/home/jovyan/logs/may-18'
writer = tf.summary.create_file_writer(logdir)


# Bracket the function call with trace_on and trace_export

tf.summary.trace_on()
# Call only one tf.function when tracing.
z = loss(x_input, y_input)
with writer.as_default():
    tf.summary.trace_export(name='graph',step=0)


In [4]:
# now add in the training bits of the graph


train_steps = 500


# learning rate
lr = 1e-1

# gradient descent optimizer
optimizer = tf.optimizers.SGD(lr)



In [9]:

with writer.as_default():
    for i in tqdm(range(train_steps)):
        with tf.GradientTape() as tape:
            current_loss = loss(x_input,y_input)
        gradients = tape.gradient(current_loss, [W, b])
        optimizer.apply_gradients(zip(gradients, [W ,b]))
        tf.summary.scalar('loss', current_loss, step=i)



  0%|          | 0/500 [00:00<?, ?it/s]

In [10]:
# now add in the training bits of the graph


train_steps = 500


# learning rate
lr = 1e-1

# gradient descent optimizer
optimizer = tf.optimizers.Adam(lr)

W = tf.Variable(tf.zeros([784,10]),name='W')
b = tf.Variable(tf.zeros([10]),name='b')

# predicted probability for each class
def y_pred(x):
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
@tf.function
def loss(x,y):
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))

In [11]:

pbar = tqdm(range(train_steps))
with writer.as_default():
    for i in pbar:
        with tf.GradientTape() as tape:
            current_loss = loss(x_input,y_input)
        gradients = tape.gradient(current_loss, [W, b])
        optimizer.apply_gradients(zip(gradients, [W ,b]))
        tf.summary.scalar('lossadam', current_loss, step=i)
        pbar.set_description_str(str(current_loss.numpy()))


  0%|          | 0/500 [00:00<?, ?it/s]

In [13]:
?tf.optimizers.Adam

Init signature:
tf.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
    **kwargs,
)
Docstring:     
Optimizer that implements the Adam algorithm.

Adam optimization is a stochastic gradient descent method that is based on
adaptive estimation of first-order and second-order moments.

According to
[Kingma et al., 2014](http://arxiv.org/abs/1412.6980),
the method is "*computationally
efficient, has little memory requirement, invariant to diagonal rescaling of
gradients, and is well suited for problems that are large in terms of
data/parameters*".

Args:
  learning_rate: A `Tensor`, floating point value, or a schedule that is a
    `tf.keras.optimizers.schedules.LearningRateSchedule`, or a callable
    that takes no arguments and returns the actual value to use, The
    learning rate. Defaults to 0.001.
  beta_1: A float value or a constant float tensor, or a callable
    that takes no arguments and returns t

In [14]:
# now add in the training bits of the graph


train_steps = 500


# learning rate
lr = 1e-1

# gradient descent optimizer
optimizer = tf.optimizers.SGD(lr)

W = tf.Variable(tf.zeros([784,10]),name='W')
b = tf.Variable(tf.zeros([10]),name='b')

# predicted probability for each class
def y_pred(x):
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
@tf.function
def loss(x,y):
    y_ = y_pred(x)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_), axis=[1]))

In [15]:
with writer.as_default():
    for i in tqdm(range(train_steps)):
        with tf.GradientTape() as tape:
            current_loss = loss(x_input,y_input)
        gradients = tape.gradient(current_loss, [W, b])
        optimizer.apply_gradients(zip(gradients, [W ,b]))

        W_slice = tf.expand_dims(tf.reshape(W[:,0],[1, 28,28]),-1)
        tf.summary.image('image', W_slice, step=i)


  0%|          | 0/500 [00:00<?, ?it/s]